# Artificial Intelligence 2023/2024

## First assignment: Informed and adversarial search strategies

### Submission: April 2, 2024
### Grading:  This assignment represents 20% of the grade for the course (4 values). If you implement more than what is requested you can get additionally a maximum of 1 value, that can complement the grade obtained globally for the project’s part.
### Authors: Alexandre Sousa (202206427), Francisco Carqueija (202205113), Guilherme Oliveira (202204987)

------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [1]:
import numpy as np
import copy
import math 
import random
import pygame
import sys


pygame 2.3.0 (SDL 2.24.2, Python 3.9.6)
Hello from the pygame community. https://www.pygame.org/contribute.html


------------------------------------------------------------------------------------------------------------------------------------------------------------------

## Connect Four: The Game

In [2]:
# Game settings
ROW_COUNT = 6
COL_COUNT = 7
SQUARESIZE = 100


class Board:
    def __init__(self):
        self.board = np.zeros((ROW_COUNT, COL_COUNT))
        self.column_heights = np.full(COL_COUNT, ROW_COUNT - 1, dtype=int)
        self.game_over = False
        self.turn = 0  # Player 1 starts

    def drop_pieces(self, player , col):
        if self.valid_col(col):
            height = self.column_heights[col]
            self.board[height][col] = player
            self.column_heights[col] = height-1
            return True
        else:
            print("Invalid move")
            return False
        
    def valid_col(self, col):
        if self.column_heights[col] == -1 :
            return False
        return True
    
    def win(self,player): 

        # Check horizontal
        for c in range(COL_COUNT-3):
            for r in range(ROW_COUNT):
                if self.board[r][c] == player and self.board[r][c+1] == player and self.board[r][c+2] == player and self.board[r][c+3] == player:
                    return True

        # Check vertical
        for c in range(COL_COUNT):
            for r in range(ROW_COUNT - 3):
                if self.board[r][c] == player and self.board[r+1][c] == player and self.board[r+2][c] == player and self.board[r+3][c] == player: 
                    return True
                
        # Check diagonal with positive slope
        for c in range(COL_COUNT - 3):
            for r in range(3,ROW_COUNT):
                if self.board[r][c] == player and self.board[r-1][c+1] == player and self.board[r-2][c+2] == player and self.board[r-3][c+3] == player: 
                    return True 
                    
        # Check diagonal with negative slope
        for c in range(COL_COUNT - 3):
            for r in range(3):
                if self.board[r][c] == player and self.board[r+1][c+1] == player and self.board[r+2][c+2] == player and self.board[r+3][c+3] == player: 
                    return True
                
        return False 

    def is_full(self):
        return np.all(self.column_heights < 0)

    def print_board(self):

        print(self.board)

In [3]:
board_aux = Board()

board_aux.drop_pieces(1,0)
board_aux.drop_pieces(1,0)
board_aux.drop_pieces(1,0)
board_aux.drop_pieces(1,0)
board_aux.drop_pieces(2,3)
board_aux.print_board()
print('\n')
print(board_aux.is_full())
print(board_aux.win(1))
print(board_aux.win(2))



[[0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 2. 0. 0. 0.]]


False
True
False


------------------------------------------------------------------------------------------------------------------------------------------------------------------

## Connect Four: Heuristic

In [4]:
class Heuristic: 
    
    def Scores(self,window,player_1, player_2):

            # Count absolute victories
            score = 0
            
            # Change to float bc thats how the numpy array is

            if np.count_nonzero(window == player_2) == 4:
                score += 100000  # Absolute victory for Player 2
                
            elif np.count_nonzero(window == player_1) == 4:
                score -= 50000  # Absolute victory for Player 1

            # Adaptation of the calculations using np.count_nonzero
            if np.count_nonzero(window == player_1) == 3 and np.count_nonzero(window == player_2) == 0: 
                score -= 500
            elif np.count_nonzero(window == player_1) == 2 and np.count_nonzero(window == player_2) == 0: 
                score -= 100
            elif np.count_nonzero(window == player_1) == 1 and np.count_nonzero(window == player_2) == 0: 
                score -= 10

            # Not necessary to treat the case of both 0, as the score does not change

            elif np.count_nonzero(window == player_1) == 0 and np.count_nonzero(window == player_2) == 1: 
                score += 10
            elif np.count_nonzero(window == player_1) == 0 and np.count_nonzero(window == player_2) == 2: 
                score += 100
            elif np.count_nonzero(window == player_1) == 0 and np.count_nonzero(window == player_2) == 3: 
                score += 500
    
            return score

    # Useful function for the first moves of the game
    def board_evaluation(self,board,player_1,player_2):

            board_score_matrix = np.array([
                [3, 4, 5, 7, 5, 4, 3],
                [4, 6, 8, 10, 8, 6, 4],
                [5, 8, 11, 13, 11, 8, 5],
                [5, 8, 11, 13, 11, 8, 5],
                [4, 6, 8, 10, 8, 6, 4],
                [3, 4, 5, 7, 5, 4, 3]
            ])
            
            player_score = 0 
            
            # Iterate over the board and calculate the score based on occupied positions
            for r in range(ROW_COUNT):
                for c in range(COL_COUNT):
                    if board[r][c] == player_1:  # Position occupied by Player 1
                        player_score -= board_score_matrix[r][c]
                    elif board[r][c] == player_2:  # Position occupied by player 2
                        player_score += board_score_matrix[r][c]
            
            return player_score

        
    # Heuristic function - 4 by 4 windows
    def evaluate_function_1(self,board,player_1,player_2):

            score = 0
            
            # Horizontally
            for r in range(ROW_COUNT):
                for c in range(COL_COUNT - 3): 
                    window = board[r][c:c+4]
                    score += self.Scores(window,player_1,player_2)
            
        
            # Vertically
            for r in range(ROW_COUNT-3):
                for c in range(COL_COUNT): 
                    window = np.array([board[r+i][c] for i in range(4)])
                    
                    score += self.Scores(window,player_1,player_2)

            # Diagonally with positive slope
            for r in range(ROW_COUNT-3):
                for c in range(COL_COUNT-1, 2, -1):
                    window = np.array([board[r+i][c-i] for i in range(4)])
                    score += self.Scores(window,player_1,player_2)

            # Diagonally with negative slope
            for r in range(ROW_COUNT-3):
                for c in range(COL_COUNT-3):
                    window = np.array([board[r+i][c+i] for i in range(4)])
                    score += self.Scores(window,player_1,player_2)

            return score
        

    def final_heuristic(self,board,player1, player2):

            eval_score = self.evaluate_function_1(board,player1,player2)
            board_score = self.board_evaluation(board,player1,player2)
            total_score = eval_score + board_score                           
            return total_score

------------------------------------------------------------------------------------------------------------------------------------------------------------------

## Connect Four:  A* Algorithm

In [5]:
heuristic = Heuristic()

In [6]:
def astar_algorithm(board, player): 
    open_list = [(0, board, None)]  # Initial cost, initial state, and no plays done yet
    best_score = float('-inf')  # Initializes the best score to negative infinity
    best_move = None  # Best move hasn't been found yet

    while open_list:

        # Remove the item with the lowest heuristic cost
        _, current_board, move = open_list.pop(0)  

        # Checks if the current movement is better than the best found so far
        current_score = heuristic.final_heuristic(current_board.board,3-player,player)
        
        if current_score > best_score:
            best_score = current_score
            best_move = move

        # If the current board represents a winning state, we do not need to continue


        # Generates the successors of the current state
        successors = generate_sucessors(current_board, player)
        

        i = 0

        for successor, succ_move in successors:

            # Calculate the heuristic cost for the successor
            heuristic_astar = heuristic.final_heuristic(successor,3-player, player)
            print(str(i) + " : " + str(heuristic_astar))
            i += 1

            # Add successor to the open list
            open_list.append((heuristic_astar, successor, succ_move))

        # Sorts the list by heuristic cost to ensure that the next state to be explored is the one with the lowest cost
        open_list.sort(key=lambda x: x[0], reverse = True)
        a,b,c = open_list.pop(0)

        return c

    # Returns the column of the best movement found
    return best_move


def generate_sucessors(board,player):
        sucessors = []
        for col in range(COL_COUNT):
            if board.valid_col(col):
                new_board = copy.deepcopy(board)
                new_board.drop_pieces(player, col)
                sucessors.append((new_board.board,col))
        return sucessors

Graphics // Observations // Conclusions

------------------------------------------------------------------------------------------------------------------------------------------------------------------

## Connect Four: MCTS Algorithm

In [7]:
C = math.sqrt(2)

class Node:
    
    def __init__(self, board, player, move = None , parent=None):
        assert isinstance(board, Board)
        self.board = board  #Board class instance
        self.parent = parent
        self.children = []
        self.move = move
        self.wins = 0
        self.visits = 0
        self.player = player

    def is_leaf(self):
        if self.board.is_full() or self.board.win(self.player):
            return True
        else:
            return False
        
    def generate_successors(self):
        successors = []
        for col in range(COL_COUNT):
            if self.board.valid_col(col):
                new_board = copy.deepcopy(self.board)
                new_board.drop_pieces(self.player, col)
                successors.append(Node(new_board, self.player, col, self))
        return successors
    
    def is_fully_expanded(self):
        possible_moves = self.generate_successors()
        return len(possible_moves) == len(self.children)
    
    def expand(self):
        possible_moves = self.generate_successors()
        for move in possible_moves: 
            self.children.append(move)


    def select_child(self):
        best_score = -float("inf")
        best_children = []
        unvisited_children = []
        for child in self.children:
            if child.visits == 0:
                unvisited_children.append(child)
            else:
                exploration_term = math.sqrt(math.log(self.visits +1) / child.visits)
                score = child.wins / child.visits + C * exploration_term
                if  score == best_score:
                    best_score = score
                    best_children = [child]
                elif score > best_score:
                    best_score = score
                    best_children = [child]
            if len(unvisited_children) > 0:
                return random.choice(unvisited_children)
        return random.choice(best_children)
    
    def backpropagate(self, result):
        self.visits += 1
        self.wins += result
        if self.parent is not None:
            self.parent.backpropagate(result)

In [8]:
def monte_carlo_tree_search(board, player, simulations=500):

    # Step 1: Initialize the tree
    root = Node(board, player)
    root.expand()  # This generates the first successors 
    
    # Step 2: Initial simulations for the first nodes
    initial_simulations_per_node = 5
    for initial_node in root.children[:7]: 
        for _ in range(initial_simulations_per_node):
            result = simulate_random_playout(initial_node.board, player)
            initial_node.backpropagate(result)
    

    for _ in range(simulations):
        node = root

        # Selection
        while not node.is_leaf():
            if node.is_fully_expanded():
                node = node.select_child()
            else:
                # Expansion
                node.expand()
                break

        # Simulation
        result = simulate_random_playout(node.board, player)

        # Backpropagation
        node.backpropagate(result)

    best_ratio = -float("inf")
    best_move = None
    for child in root.children:
        if child.visits > 0:
            ratio = child.wins / child.visits
            print(ratio)
        else:
            ratio = 0
        if ratio > best_ratio:
            best_ratio = ratio
            best_move = child.move
    
    # Returns the movement of the best child
    return best_move
    
    
def simulate_random_playout(game_state, player):
    simulated_game = copy.deepcopy(game_state)
    current_player = player

    while not simulated_game.is_full() and not simulated_game.win(current_player):
        possible_moves = [col for col in range(COL_COUNT) if simulated_game.valid_col(col)]
        move = random.choice(possible_moves)
        simulated_game.drop_pieces(current_player, move)
        current_player = 1 if current_player == 2 else 2  # Switch player
    
    if simulated_game.win(current_player):
        return 1 if current_player == player else 0
    else:
        return 0.5  # Consider draw as half a win


Graphics // Observations // Conclusions

------------------------------------------------------------------------------------------------------------------------------------------------------------------

## Connect Four: Minimax Algorithm

In [9]:
def minimax(board, depth, player_1, player_2, current_player, alpha=float('-inf'), beta=float('inf')):
    
    maximizing_player = current_player == player_2
    

    if depth == 0 or board.is_full() or board.win(current_player):
        score = heuristic.final_heuristic(board.board, player_1, player_2)
        return score, -1
        
    if maximizing_player:
        best = float('-inf')
        best_move = -1
        
        for col in range(COL_COUNT):
            if board.valid_col(col):
                new_board = copy.deepcopy(board)
                new_board.drop_pieces(current_player, col)
                eval, _ = minimax(new_board, depth - 1, player_1, player_2, player_1, alpha, beta)
                
                if eval > best:
                    best_move = col
                    best = eval
                
                alpha = max(alpha, best)
                
                if alpha >= beta:
                    break
                    
        return best, best_move
    else:
        best = float('inf')
        best_move = -1
        
        for col in range(COL_COUNT):
            if board.valid_col(col):
                new_board = copy.deepcopy(board)
                new_board.drop_pieces(current_player, col)
                eval, _ = minimax(new_board, depth - 1, player_1, player_2, player_2, alpha, beta)
                
                if eval < best:
                    best_move = col
                    best = eval
                
                beta = min(beta, best)
                
                
                if beta <= alpha:
                    
                    break
                    
        return best, best_move

Graphics // Observations // Conclusions

------------------------------------------------------------------------------------------------------------------------------------------------------------------

## Connect Four: Negamax Algorithm

In [10]:
def negamax(board, depth, player, alpha=float('-inf'), beta=float('inf')):
    if depth == 0 or board.is_full() or board.win(player):
        
        return heuristic.final_heuristic(board.board, 1, 2) * (-1 if player == 1 else 1), -1

    max_eval = float('-inf')
    player_move = -1

    for col in range(COL_COUNT):
        if board.valid_col(col):
            new_board = copy.deepcopy(board)
            new_board.drop_pieces(player, col)
            eval, _ = negamax(new_board, depth - 1, 3 - player, -beta, -alpha)
            eval = -eval  
            
            if eval > max_eval:
                max_eval = eval
                player_move = col

            alpha = max(alpha, eval)
            if alpha >= beta:
                break

    return max_eval, player_move

Graphics // Observations // Conclusions

------------------------------------------------------------------------------------------------------------------------------------------------------------------

## Connect Four:  Game Interface

In [11]:
# Initialize Pygame
pygame.init()

# Define colors
BLUE = (0, 0, 255)
BLACK = (0, 0, 0)
RED = (255, 0, 0)
YELLOW = (255, 255, 0)

width = COL_COUNT * SQUARESIZE
height = (ROW_COUNT + 1) * SQUARESIZE
size = (width, height)

RADIUS = int(SQUARESIZE / 2 - 5)

screen = pygame.display.set_mode(size)

def draw_board(board):
    for c in range(COL_COUNT):
        for r in range(ROW_COUNT):
            pygame.draw.rect(screen, BLUE, (c * SQUARESIZE, (r+1) * SQUARESIZE, SQUARESIZE, SQUARESIZE))
            pygame.draw.circle(screen, BLACK, (int(c * SQUARESIZE + SQUARESIZE / 2), int((r+1) * SQUARESIZE + SQUARESIZE / 2)), RADIUS)
    
    for c in range(COL_COUNT):
        for r in range(ROW_COUNT):
            if board.board[r][c] == 1:
                pygame.draw.circle(screen, RED, (int(c * SQUARESIZE + SQUARESIZE / 2),  int((r+1) * SQUARESIZE + SQUARESIZE / 2)) , RADIUS)
            elif board.board[r][c] == 2:
                pygame.draw.circle(screen, YELLOW, (int(c * SQUARESIZE + SQUARESIZE / 2), int((r+1)* SQUARESIZE + SQUARESIZE / 2))  , RADIUS)
    pygame.display.update()




class Menu: 
    def draw_menu(self,screen):
        screen.fill((200, 200, 200))  # Sets the screen background color Gray

        # Sets the title font to a different font and draws the title
        title_font = pygame.font.SysFont("comicsansms", 60)  # Altera para "comicsansms" e aumenta o tamanho
        title_text = title_font.render("Connect 4", True, (0, 0, 0))  # Cor do texto do título (preto)
        title_rect = title_text.get_rect(center=(width // 2, 50))
        screen.blit(title_text, title_rect)

        # Configurações para os modos de jogo
        font = pygame.font.SysFont("Arial", 36)  # Fonte para os modos de jogo
        menu_bg_color = (70, 70, 70)  # Altera para cinza escuro
        text_color = (255, 255, 255)  # Cor do texto (branco)
        modes = ["Player vs Player", "Player vs CPU", "CPU vs CPU"]
        mode_rects = []

        for i, mode in enumerate(modes):
            # Calcula a posição e tamanho do retângulo para cada modo
            rect_x = (width - (width // 2)) // 2  # Centraliza o retângulo
            rect_y = 150 + i * 100 - 10
            rect_width = width // 2
            rect_height = 60

            # Desenha o retângulo de fundo para cada modo de jogo
            mode_rect = pygame.Rect(rect_x, rect_y, rect_width, rect_height)
            pygame.draw.rect(screen, menu_bg_color, mode_rect)
            mode_rects.append(mode_rect)

            # Renderiza e desenha o texto do modo de jogo sobre o retângulo
            text = font.render(mode, True, text_color)
            text_rect = text.get_rect(center=(width // 2, 150 + i * 100))
            screen.blit(text, text_rect)

        pygame.display.update()
        return mode_rects


    def menu_screen(self):

        running = True
        mode_rects = self.draw_menu(screen)

        while running:
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    running = False
                    sys.exit()
                elif event.type == pygame.MOUSEBUTTONDOWN:
                    x, y = event.pos
                    for i, rect in enumerate(mode_rects):
                        if rect.collidepoint(x, y):
                            return i  # Retorna o índice do modo selecionado


            pygame.display.update()

    def algorithm_screen(self, message):
            running = True
            mode_rects = self.draw_algorithm_menu(screen, message)

            while running:
                for event in pygame.event.get():
                    if event.type == pygame.QUIT:
                        running = False
                        sys.exit()
                    elif event.type == pygame.MOUSEBUTTONDOWN:
                        x, y = event.pos
                        for i, rect in enumerate(mode_rects):
                            if rect.collidepoint(x, y):
                                return i  # Retorna o índice do modo selecionado

                pygame.display.update()

    def draw_algorithm_menu(self, screen, message):
        screen.fill((200, 200, 200))  # Fundo

        # Título do submenu
        title_font = pygame.font.SysFont("comicsansms", 40)
        title_text = title_font.render(message , True, (0, 0, 0))
        title_rect = title_text.get_rect(center=(width // 2, 50))
        screen.blit(title_text, title_rect)

        # Opções de algoritmo
        algorithms = ["A*", "Monte Carlo", "Minimax", "Negamax"]
        algorithm_rects = []

        for i, algorithm in enumerate(algorithms):
            rect_x = (width - (width // 3)) // 2
            rect_y = 150 + i * 100
            rect_width = width // 3
            rect_height = 50

            # Desenha retângulo para cada algoritmo
            algorithm_rect = pygame.Rect(rect_x, rect_y, rect_width, rect_height)
            pygame.draw.rect(screen, (70, 70, 70), algorithm_rect)
            algorithm_rects.append(algorithm_rect)

            # Texto para cada algoritmo
            font = pygame.font.SysFont("Arial", 28)
            text = font.render(algorithm, True, (255, 255, 255))
            text_rect = text.get_rect(center=(width // 2, 150 + i * 100))
            screen.blit(text, text_rect)

        pygame.display.update()

        return algorithm_rects


2024-03-29 04:43:08.601 Python[10876:11730453] WARNING: Secure coding is automatically enabled for restorable state! However, not on all supported macOS versions of this application. Opt-in to secure coding explicitly by implementing NSApplicationDelegate.applicationSupportsSecureRestorableState:.


------------------------------------------------------------------------------------------------------------------------------------------------------------------

## Connect Four

In [12]:
board = Board()
Inicio = Menu()
mode_index = Inicio.menu_screen()

if mode_index == 1 :
    algorithm_index = Inicio.algorithm_screen("Escolha o Algoritmo") 
  
elif mode_index == 2:
    algorithm_index = Inicio.algorithm_screen("Escolha o 1º Algoritmo") 
    print(algorithm_index)
    algorithm_index2 = Inicio.algorithm_screen(" Agora escolha o 2º Algoritmo e aguarde...")
    print(algorithm_index2)

alternate = 0

Inicio.draw_algorithm_menu(screen, " ")
  

# Main game loop
while not board.game_over:
    
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            pygame.quit()
            sys.exit()

        if mode_index == 0:  # Player vs Player
            if event.type == pygame.MOUSEBUTTONDOWN:
                x_pos = event.pos[0]
                col = int(x_pos // SQUARESIZE)
                if board.drop_pieces(board.turn + 1, col):
                    if board.win(board.turn + 1):
                        print(f"Player {board.turn + 1} wins!")
                        board.game_over = True
                    board.turn = 1 - board.turn  # Switch turns

        elif mode_index == 1:  # Player vs CPU
            if board.turn == 0:  # Player's turn
                if event.type == pygame.MOUSEBUTTONDOWN:
                    x_pos = event.pos[0]
                    col = int(x_pos // SQUARESIZE)
                    if board.drop_pieces(1, col):
                        if board.win(1):
                            print("Player 1 wins!")
                            board.game_over = True
                        board.turn = 1 - board.turn  # Switch to CPU's turn
            else:  # CPU's turn
                # Define player_1 and player_2 for clarity
                player_1 = 1
                player_2 = 2
                current_player = board.turn + 1  # This adjusts the player number correctly for the Minimax call

                # Select the column based on the algorithm
                if algorithm_index == 0:
                    col = astar_algorithm(board, current_player)
                elif algorithm_index == 1:
                    col = monte_carlo_tree_search(board, current_player, simulations=5000)
                elif algorithm_index == 2:
                    _, col = minimax(board, 5, player_1, player_2, current_player)
                elif algorithm_index == 3:
                    col = negamax(board, 7, current_player == player_2, player_1, player_2, current_player)[1]

                if board.drop_pieces(current_player, col):
                    if board.win(current_player):
                        print(f"CPU {current_player} wins!")
                        board.game_over = True
                    board.turn = 1 - board.turn  # Switch back to the player's turn

        draw_board(board)

        if board.is_full():
            print("The game is a draw!")
            board.game_over = True
            break

        pygame.display.update()

    if mode_index == 2:  # CPU vs CPU
            pygame.time.wait(2000)  # Add a small delay to make moves visible

            # Define player_1 and player_2 for clarity, assuming player_1 is CPU 1 and player_2 is CPU 2
            player_1 = 1
            player_2 = 2
            current_player = board.turn + 1

            # Select the current CPU's algorithm
            algorithm = algorithm_index if board.turn == 0 else algorithm_index2

            # Select the column based on the selected algorithm
            if algorithm == 0:
                col = astar_algorithm(board, current_player)
            elif algorithm == 1:
                col = monte_carlo_tree_search(board, current_player, simulations=5000)
            elif algorithm == 2 and current_player == 1:
                _, col = minimax(board, 5, player_1, player_2, current_player)

            elif algorithm == 2 and current_player == 2:
                _, col = minimax(board, 5, player_2,player_1, current_player)

            elif algorithm == 3:
                col = negamax(board, 7, current_player == player_2, player_1, player_2, current_player)[1]

            print(f"BOARD TURN: {board.turn}, Column selected by CPU {current_player}: {col}")

            if board.drop_pieces(current_player, col):
                if board.win(current_player):
                    print(f"CPU {current_player} wins!")
                    board.game_over = True
                board.turn = 1 - board.turn  # Switch turns between CPUs

            draw_board(board)

            if board.is_full():
                print("The game is a draw!")
                board.game_over = True
                break

            pygame.display.update()

        

pygame.time.wait(3000)  # Waits a bit before closing the game


pygame.quit()

Player 1 wins!
